To use this Notebook, follow the instructions in [README.md](../../git/README.md)

In [1]:
import pandas as pd               # a data.frame handler like R
import folium                     # displaying maps
import os
import matplotlib.pyplot as plt
import osmnx as ox                # connection to OpenStreetMap
from selenium import webdriver    # for rendering in browser to save as .png
import time

In [2]:
# read data
current_file = globals()['_dh'][0]  # absolut path of this file working in jupyter notebook
df = pd.read_table(current_file + os.sep + 'data' + os.sep + 'trafficCounts.csv', sep =';')  # os.sep as os inedpendent sep
#df.info()
#df.head()

In [3]:
tmp = df[df['DateTimeFrom'] == '2019-01-01T00:00:00+01:00'] # select data from that date
len(tmp) # number of rows
tmp['SiteCode'].unique() 
tmp = tmp[tmp['SiteCode'] < 1000]  # siteCode are integers, to use string length, .str.len()
agg = tmp.groupby('SiteCode').PW.agg('sum')   # aggregate PW
f = pd.merge(agg, tmp[['SiteCode','Geo Point']], how='inner',on='SiteCode')
f['SiteCode'].unique().tolist()  # list of unique entries in col SiteCode
# split Geo Point in lng an lat
f[['lat','lng']] = f['Geo Point'].str.split(',', expand=True) # split Geo Points into new cols
# for calculation circles
maxPW = f['PW'].max()
minPW = f['PW'].min()
meanPW = f['PW'].mean()


In [5]:
def plotDot(row):
    radius = row.PW / maxPW * 10
    folium.CircleMarker(location=[row.lat, row.lng],radius=radius,color = '#f44336',fill=True, fillcolor = '#f44336', fill_opacity =1).add_to(m)

# the map
place_name = 'Basel, Switzerland'
graph = ox.graph_from_place(place_name)
nodes, streets = ox.graph_to_gdfs(graph)
#streets.head()
style = {'color': '#F7DC6F', 'weight':'1'}
m = folium.Map([47.561195,7.581742], width=900, height =650,zoom_start=14, tiles='CartoDB Positron')
# plot points
f.apply(plotDot, axis = 1)
m


In [ ]:
# Save as map as image
# download gecko driver for firefox from here - https://github.com/mozilla/geckodriver/releases
mapFname = 'output.html'
m.save(mapFname)
mapUrl = 'file://{0}/{1}'.format(os.getcwd(), mapFname)
# use selenium webdriver to save the html as png image
driver = webdriver.Firefox()
driver.get(mapUrl)
# wait for 5 seconds for the maps and other assets to be loaded in the browser
time.sleep(5)
driver.save_screenshot('output.png')
driver.quit()